# Кластеризация товаров по их описанию(только ИМпорт)


### Данные храняться в файле 3208.xlsx. Само описание в G31_1_description = 'G31_1 (Наименование и характеристики товаров)'. Для кластеризации используеться метод определения TF-IDF векторов для текста описания товара. 
Перед векторизацией данные группируются по описанию и удалаються все знаки препинания. В процессе векторизации пременяеться токенизация и стемминг.
После получения ветора для кластеризации применяеться метод К-средних(со случайно выбранным числом кластеров - 25) и 
метод DBSCAN. Стоп-слова по русскому языку храняться в файле "stopwords-ru.txt"

In [7]:
import numpy as np
import pandas as pd
import nltk
import re
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.stem.snowball import SnowballStemmer

G31_1_description = 'G31_1 (Наименование и характеристики товаров)'
G011_direction = 'G011 (Направление перемещения)' #импорт - экспорт
G022_name_sender = 'G022 (Наименование отправителя)'
G080_recipient_OGRN = 'G080 (ОГРН получателя)'
G081_recipient_INN = 'G081 (ИНН получателя)'
G082_recipient_Name = 'G082 (Наименование получателя)'
G221_currency = 'G221 (Букв.код валюты контракта)'
G35_gross_weight = 'G35 (Вес брутто, кг)'
G38_net_weight = 'G38 (Вес нетто, кг)'
G12_customs_value_total = 'G12 (Общая таможенная стоимость по ГТД)'
G222_invoice_value_total = 'G222 (Общая фактурная стоимость по ГТД)'
G23_exchange_rates = 'G23 (Курс валюты)'
G42_invoice_value = 'G42 (Фактурная стоимость)'
G42RUB_invoice_value_rub = 'G42RUB (Фактурная стоимость в рублях)'
G45_customs_value = 'G45 (Таможенная стоимость)'
G46_statistic_value = 'G46 (Статистическая стоимость, USD.)'
usd_kg = 'USDKG (USD за КГ)'
G30_city_warehouse = 'G30CITY (Город склада)'

#нужные нам стольбцы
columns = [G31_1_description, G022_name_sender, G080_recipient_OGRN, G081_recipient_INN, 
           G082_recipient_Name, G221_currency, G23_exchange_rates, 
           G30_city_warehouse, G38_net_weight, G42_invoice_value, G42RUB_invoice_value_rub, G46_statistic_value, usd_kg]

#функция токенизации и стемминга
def token_and_stem(text):
    tokens  = [word for word in nltk.word_tokenize(text)] # выделяем слова
    #print(tokens, '\n')
    filt_tokens = []
    for token in tokens:
        if re.search('[а-яА-Яa-zA-Z]', token): #убираем цифры
            if len(token) > 3: #смотрим, что бы слова состояли минимум из 4 символов
                filt_tokens.append(token)
    #print(filt_tokens, '\n')
    stems = [] #стемминг
    for token in filt_tokens:
        if re.search('[а-яА-Я]', token): #для русских слов стемминг
            stems.append(stemmer_rus.stem(token))
        elif re.search('[a-zA-Z]', token):#для английских слов стемминг
            stems.append(stemmer_eng.stem(token))
    #print(stems)
    return stems

df_excel = pd.read_excel('3208.xlsx')#считываем данные в датафрейм

mask = df_excel[G011_direction] == 'ИМ'#выбираем направление ИМпорт
df = df_excel.copy()[mask]
df = df[columns]
df = df.groupby(G31_1_description).aggregate(sum)#группируем данные
df = df.reset_index()
df[G31_1_description] = df[G31_1_description].str.lower() #нижний регистр
# начинаем отделять знаки препинания
df[G31_1_description] = df[G31_1_description].str.replace(',', ' ') 
df[G31_1_description] = df[G31_1_description].str.replace('.', ' ') 
df[G31_1_description] = df[G31_1_description].str.replace('-', ' ')
df[G31_1_description] = df[G31_1_description].str.replace(';', ' ')
df[G31_1_description] = df[G31_1_description].str.replace(':', ' ')
df[G31_1_description] = df[G31_1_description].str.replace('(', ' ')
df[G31_1_description] = df[G31_1_description].str.replace(')', ' ')
df[G31_1_description] = df[G31_1_description].str.replace(r'[\W]+', ' ')
# в тексте используеться английский и русский язык
stemmer_rus = SnowballStemmer('russian')
stemmer_eng = SnowballStemmer('english')

#создаем стоп слова из англ и русского алфавита
file_name_stopwords = 'stopwords-ru.txt'
f = open(file_name_stopwords, encoding='utf8').read()
stopwords_rus = f.split('\n')
stopwords_eng = nltk.corpus.stopwords.words('english')
stopwords = []
stopwords.extend(stopwords_rus)
stopwords.extend(stopwords_eng)
#векторизация
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
tfidf_vectorizer = TfidfVectorizer(max_df= 0.9, max_features= 1000, min_df=0.05, stop_words= stopwords, 
                                  use_idf= True, tokenizer=token_and_stem, ngram_range=(1,5))
tfidf_matrix = tfidf_vectorizer.fit_transform(df[G31_1_description])



C:\Users\user\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'becaus', 'befor', 'could', 'doe', 'dure', 'might', 'must', 'need', 'onc', 'onli', 'ourselv', 'themselv', 'veri', 'would', 'yourselv', 'алл', 'бел', 'близк', 'бол', 'больш', 'буд', 'будеш', 'будт', 'быв', 'быва', 'быт', 'важн', 'вдал', 'ве', 'вед', 'везд', 'вернут', 'взят', 'видет', 'вмест', 'вод', 'войн', 'вообщ', 'восем', 'восемнадца', 'восемнадцат', 'восьм', 'впроч', 'врем', 'времен', 'всег', 'всегд', 'всюд', 'втор', 'выйт', 'главн', 'говор', 'голов', 'дава', 'давн', 'даж', 'далек', 'дальш', 'дар', 'дат', 'двадца', 'двадцат', 'двенадца', 'двенадцат', 'двер', 'девя', 'девят', 'девятнадца', 'девятнадцат', 'действительн', 'дела', 'ден', 'деньг', 'деся', 'десят', 'довольн', 'долг', 'долж', 'должн', 'дорог', 'дума', 'душ', 'есл', 'ест', 'ждат', 'жен', 'женщин', 'жизн', 'жит', 'зан', 

# Метод k-средних


In [10]:
num_clusters = 25 #выбираем наугад число кластеров
from sklearn.cluster import KMeans
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf_matrix)
clusters = km.labels_.tolist()
clust = km.predict(tfidf_matrix)
df_Kmeans = df.copy()
df_Kmeans['Cluster'] = clusters # создаем и записываем в датафрейм кластеры
df_Kmeans.to_excel('3208_cluster_Kmeans_25.xlsx')


# DBSCAN

In [11]:
from sklearn.cluster import DBSCAN
db = DBSCAN(eps = 0.3, min_samples= 10).fit(tfidf_matrix)
labels = db.labels_.tolist()
df_db = df.copy()
df_db['Cluster'] = labels # создаем и записываем в датафрейм кластеры
df_db.to_excel('3208_cluster_DBSCAN.xlsx')